In [27]:
import importlib

# ENVIRONMENT
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import dotenv
import os

dotenv.load_dotenv('.env')
MT5_SERVER = os.environ["MT5_SERVER"]
MT5_LOGIN = os.environ["MT5_LOGIN"]
MT5_PASSWORD = os.environ["MT5_PASSWORD"]
DATA_PATH = os.environ["DATA_PATH"]
CATALOG_PATH = os.path.join(os.getcwd(), os.environ["CATALOG_PATH"])


[autoreload of strategies.bollinger_cluster failed: Traceback (most recent call last):
  File "/Users/tobiaspucher/GitHub/nautilus/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/tobiaspucher/GitHub/nautilus/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/tobiaspucher/GitHub/nautilus/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/Users/tobiaspucher/GitHub/nautilus/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 365, in update_class
    update_instances(old, new)
  File "/Users/tobiaspucher/GitHub/nautilus/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 323, in update_instances
    object.__setattr__(ref, "__class__", new)
TypeError: can't apply this __setattr__

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [28]:
# nautilus_trader imports

from nautilus_trader.model.identifiers import Venue, InstrumentId, Symbol
from nautilus_trader.model.data import Bar, BarType, QuoteTick
from nautilus_trader.config import BacktestVenueConfig, BacktestDataConfig, BacktestRunConfig, BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.engine import BacktestResult
from nautilus_trader.trading.strategy import ImportableStrategyConfig
from nautilus_trader.core.datetime import dt_to_unix_nanos, maybe_unix_nanos_to_dt, unix_nanos_to_dt
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.cache.cache import Cache
from nautilus_trader.model.position import Position
from nautilus_trader.model.objects import Price


from decimal import Decimal
from datetime import datetime

# other imports
from pandas import Timestamp
import importlib
import mplfinance as mpf
import matplotlib.pyplot as plt

# my packages
import indicators
from indicators import TrackerFloat, TrackerMulti

import strategies.bollinger_cluster
from strategies.bollinger_cluster import BollingerCluster
import data_utils
import utils
importlib.reload(indicators)
importlib.reload(strategies.bollinger_cluster)
importlib.reload(data_utils)
importlib.reload(utils)


catalog = ParquetDataCatalog(CATALOG_PATH)

start = dt_to_unix_nanos(pd.Timestamp("2023-11-01 00:00:00"))
end = start + pd.Timedelta(days=30).value 

venue_str = "SIM_EIGHTCAP"
venue = Venue(venue_str)
symbol_str = "EURUSD"
symbol = Symbol(symbol_str)
instrument_id_str = f"EURUSD.{venue}"

instrument_id = InstrumentId(symbol, venue)

<module 'indicators' from '/Users/tobiaspucher/GitHub/nautilus/indicators.py'>

<module 'strategies.bollinger_cluster' from '/Users/tobiaspucher/GitHub/nautilus/strategies/bollinger_cluster.py'>

<module 'data_utils' from '/Users/tobiaspucher/GitHub/nautilus/data_utils.py'>

<module 'utils' from '/Users/tobiaspucher/GitHub/nautilus/utils.py'>

In [29]:
venue_configs = [
    BacktestVenueConfig(
        name=venue_str,
        oms_type="HEDGING",
        account_type="MARGIN",
        base_currency="USD",
        starting_balances=["10_000 USD"],
    ),
]

data_configs = [
    BacktestDataConfig(
        catalog_path=CATALOG_PATH,
        data_cls=QuoteTick,
        instrument_id=instrument_id,
        start_time=start,
        end_time=end,
    ),
]

strategies = [
    ImportableStrategyConfig(
        strategy_path="strategies.bollinger_cluster:BollingerCluster",
        config_path="strategies.bollinger_cluster:BollingerClusterConfig",
        config=dict(
            instrument_id=instrument_id.value,
            bar_type=f"{instrument_id}-15-MINUTE-BID-INTERNAL",
            bb_params=[
                (5, 2),
                (60, 2)
            ],
        ),
    ),
]

configs = [BacktestRunConfig(
    engine=BacktestEngineConfig(strategies=strategies),
    data=data_configs,
    venues=venue_configs,
)]

node = BacktestNode(configs)
print(strategies)
print(node)

[ImportableStrategyConfig(strategy_path='strategies.bollinger_cluster:BollingerCluster', config_path='strategies.bollinger_cluster:BollingerClusterConfig', config={'instrument_id': 'EURUSD.SIM_EIGHTCAP', 'bar_type': 'EURUSD.SIM_EIGHTCAP-15-MINUTE-BID-INTERNAL', 'bb_params': [(5, 2), (60, 2)]})]


In [30]:
results = node.run()
res = results[0]
backtest_start = maybe_unix_nanos_to_dt(res.backtest_start)
backtest_end = maybe_unix_nanos_to_dt(res.backtest_end)

Failed to set global default dispatcher because of error: a global default trace dispatcher has already been set

2024-01-19T12:24:58.229602001Z [INF] BACKTESTER-001.BacktestEngine: =================================================================
initialized strategy
2024-01-19T12:24:58.229607001Z [INF] BACKTESTER-001.BacktestEngine:  NAUTILUS TRADER - Automated Algorithmic Trading Platform
2024-01-19T12:24:58.229607002Z [INF] BACKTESTER-001.BacktestEngine:  by Nautech Systems Pty Ltd.
2024-01-19T12:24:58.229608001Z [INF] BACKTESTER-001.BacktestEngine:  Copyright (C) 2015-2023. All rights reserved.
2024-01-19T12:24:58.229608002Z [INF] BACKTESTER-001.BacktestEngine: =================================================================
2024-01-19T12:24:58.229608003Z [INF] BACKTESTER-001.BacktestEngine: 
2024-01-19T12:24:58.229609001Z [INF] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣴⣶⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2024-01-19T12:24:58.229609002Z [INF] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⣾⣿⣿⣿⠀⢸⣿⣿⣿⣿

2023-11-02T03:00:00.768000000Z [ERR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from book when fills were expected (check sizes in data).


-29, venue_order_id=None, position_id=None, contingency_type=OUO, linked_order_ids=[O-20231102-0300-001-000-30], parent_order_id=O-20231102-0300-001-000-28, tags=STOP_LOSS), LimitOrder(BUY 1_000 EURUSD.SIM_EIGHTCAP LIMIT @ 1.05769 GTC, status=INITIALIZED, client_order_id=O-20231102-0300-001-000-30, venue_order_id=None, position_id=None, contingency_type=OUO, linked_order_ids=[O-20231102-0300-001-000-29], parent_order_id=O-20231102-0300-001-000-28, tags=TAKE_PROFIT)]), position_id=None).
2023-11-02T03:00:00.000000000Z [INF] BACKTESTER-001.BollingerCluster-000: <--[EVT] OrderSubmitted(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231102-0300-001-000-28, account_id=SIM_EIGHTCAP-001, ts_event=1698894000000000000).
2023-11-02T03:00:00.000000000Z [INF] BACKTESTER-001.BollingerCluster-000: <--[EVT] OrderSubmitted(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231102-0300-001-000-29, account_id=SIM_EIGHTCAP-001, ts_event=1698894000000000000).
2023-11-02T03:00:00.000000000Z [IN

2023-11-06T10:00:03.424000000Z [ERR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from book when fills were expected (check sizes in data).


tenance=0.00 USD, instrument_id=EURUSD.SIM_EIGHTCAP)], event_id=6601821a-fed7-4606-b780-58e357ffc1be).
2023-11-06T01:30:00.000000000Z [INF] BACKTESTER-001.BollingerCluster-000: <--[EVT] OrderAccepted(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231106-0130-001-000-109, venue_order_id=SIM_EIGHTCAP-1-013, account_id=SIM_EIGHTCAP-001, ts_event=1699234200000000000).
2023-11-06T01:30:00.000000000Z [INF] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Pending OTO O-20231106-0130-001-000-110 triggers from O-20231106-0130-001-000-109
2023-11-06T01:30:00.000000000Z [INF] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Pending OTO O-20231106-0130-001-000-111 triggers from O-20231106-0130-001-000-109
2023-11-06T01:30:31.907000000Z [INF] BACKTESTER-001.Portfolio: EURUSD.SIM_EIGHTCAP margin_init=32.23 USD
2023-11-06T01:30:31.907000000Z [INF] BACKTESTER-001.Portfolio: Updated AccountState(account_id=SIM_EIGHTCAP-001, account_type=MARGIN, base_currency=USD, is_reported=False, balances=[

...

=TAKE_PROFIT)]), position_id=None).
2023-11-09T11:45:00.000000000Z [INF] BACKTESTER-001.BollingerCluster-000: <--[EVT] OrderSubmitted(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231109-1145-001-000-202, account_id=SIM_EIGHTCAP-001, ts_event=1699530300000000000).
2023-11-09T11:45:00.000000000Z [INF] BACKTESTER-001.BollingerCluster-000: <--[EVT] OrderSubmitted(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231109-1145-001-000-203, account_id=SIM_EIGHTCAP-001, ts_event=1699530300000000000).
2023-11-09T11:45:00.000000000Z [INF] BACKTESTER-001.BollingerCluster-000: <--[EVT] OrderSubmitted(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231109-1145-001-000-204, account_id=SIM_EIGHTCAP-001, ts_event=1699530300000000000).
2023-11-09T11:45:00.000000000Z [WRN] BACKTESTER-001.BollingerCluster-000: <--[EVT] OrderRejected(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231109-1145-001-000-202, account_id=SIM_EIGHTCAP-001, reason='LIMIT_IF_TOUCHED BUY order trigger s

2023-11-13T14:15:00.171000000Z [ERR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from book when fills were expected (check sizes in data).
2023-11-14T09:30:23.457000000Z [ERR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from book when fills were expected (check sizes in data).
2023-11-15T11:15:09.306000000Z [ERR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from book when fills were expected (check sizes in data).


mulation_trigger=NO_TRIGGER, trigger_instrument_id=None, contingency_type=OTO, order_list_id=OL-20231115-0115-001-000-103, linked_order_ids=['O-20231115-0115-001-000-308', 'O-20231115-0115-001-000-309'], parent_order_id=None, exec_algorithm_id=None, exec_algorithm_params=None, exec_spawn_id=None, tags=ENTRY).
2023-11-15T01:15:00.000000000Z [INF] BACKTESTER-001.BollingerCluster-000: <--[EVT] OrderInitialized(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231115-0115-001-000-308, side=BUY, type=STOP_MARKET, quantity=1_000, time_in_force=GTC, post_only=False, reduce_only=True, quote_quantity=False, options={'trigger_price': '1.09310', 'trigger_type': 'DEFAULT', 'expire_time_ns': 0}, emulation_trigger=NO_TRIGGER, trigger_instrument_id=None, contingency_type=OUO, order_list_id=OL-20231115-0115-001-000-103, linked_order_ids=['O-20231115-0115-001-000-309'], parent_order_id=O-20231115-0115-001-000-307, exec_algorithm_id=None, exec_algorithm_params=None, exec_spawn_id=None, tags=STOP_LO

2023-11-15T15:45:00.875000000Z [ERR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from book when fills were expected (check sizes in data).


231115-1545-001-000-337, account_id=SIM_EIGHTCAP-001, ts_event=1700063100000000000).
2023-11-15T15:45:00.000000000Z [INF] BACKTESTER-001.BollingerCluster-000: <--[EVT] OrderSubmitted(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231115-1545-001-000-338, account_id=SIM_EIGHTCAP-001, ts_event=1700063100000000000).
2023-11-15T15:45:00.000000000Z [INF] BACKTESTER-001.BollingerCluster-000: <--[EVT] OrderSubmitted(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231115-1545-001-000-339, account_id=SIM_EIGHTCAP-001, ts_event=1700063100000000000).
2023-11-15T15:45:00.000000000Z [INF] BACKTESTER-001.Portfolio: EURUSD.SIM_EIGHTCAP margin_init=32.57 USD
2023-11-15T15:45:00.000000000Z [INF] BACKTESTER-001.Portfolio: Updated AccountState(account_id=SIM_EIGHTCAP-001, account_type=MARGIN, base_currency=USD, is_reported=False, balances=[AccountBalance(total=10_002.80 USD, locked=32.57 USD, free=9_970.23 USD)], margins=[MarginBalance(initial=32.57 USD, maintenance=0.00 USD, instrument_id

In [31]:
res

BacktestResult(trader_id='BACKTESTER-001', machine_id='client-145-100-229-030.erdg.usor.nl', run_config_id='a85e667f3c3936ee5dff1e1265ee193bc902d1ee0b40fdf41cfa34dca3e905fc', instance_id='c7abbe5d-b2a8-44f5-8d0f-81f6a10ec3db', run_id='3a0a066b-b39f-4a27-984c-f7f34136c8a3', run_started=1705667099619833001, run_finished=1705667106478225001, backtest_start=1698796800065000000, backtest_end=1701388799630000000, elapsed_time=2591999.565, iterations=0, total_events=1755, total_orders=813, total_positions=26, stats_pnls={'USD': {'PnL (total)': -4.6, 'PnL% (total)': -0.04600000000000364, 'Max Winner': 1.14, 'Avg Winner': 0.9780952380952382, 'Min Winner': 0.96, 'Min Loser': -5.01, 'Avg Loser': -5.028, 'Max Loser': -5.04, 'Expectancy': -0.17692307692307663, 'Win Rate': 0.8076923076923077}}, stats_returns={'Returns Volatility (252 days)': 0.036436757423573496, 'Average (Return)': -0.00012348105856434675, 'Average Loss (Return)': -0.004606384900328707, 'Average Win (Return)': 0.0009438769989985963

In [32]:
# This allows multiple outputs from a single jupyter notebook cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib qt


engine = node.get_engine(res.run_config_id)
strategy: BollingerCluster = engine.trader.strategies()[0]
cache: Cache = strategy.cache

layout = get_layout(bars=strategy.bars,
                    line_indicators=strategy.trackers,
                    positions=strategy.cache.positions(),
                    picker_start=backtest_start, 
                    picker_end=backtest_end)

# standalone HTML file
reset_output()
show(layout)

In [9]:
import mplfinance as mpf

bars = strategy.bars
df = utils.df_from_bars(bars)

MPF_MAIN_PANEL_IDX = 0

addplots = []

add_plot_dict = [mpf.make_addplot(t.get_df(), panel=MPF_MAIN_PANEL_IDX, ylabel="Bollinger Bands", color="blue", alpha=0.2) for t in strategy.trackers]
addplots.extend(add_plot_dict)

mpf.plot(df, addplot=addplots, type="candle", style="charles", volume=True, figratio=(16,9), figscale=1.5)

def positions_to_markers(positions: list[Position], bars: list[Bar]):
    open_markers = []
    close_markers = []
    for b in bars:
        for p in positions:
            #position opened within this bar
            if p.ts_opened >= b.ts_start and p.ts_opened <= b.ts_end:
                open_markers.append(p.avg_px_open)
            else:
                open_markers.append(None)
                
            #position closed within this bar
            if b.ts_start <= p.ts_closed <= b.ts_end:
                close_markers.append(p.avg_px_close)
            else:
                close_markers.append(None)
            
            

/Users/tobiaspucher/GitHub/nautilus/.venv/lib/python3.11/site-packages/mplfinance/_arg_validators.py:84: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+


ValueError: x and y must have same first dimension, but have shapes (2052,) and (2092,)